In [40]:
import pandas as pd
import geopandas as gpd


# Load geospatial data for Berlin postal codes
df_geodat_plz = pd.read_csv("../data/geodata_berlin_plz.csv", delimiter=";")

# Load electric charging station data from an Excel file
df_lstat = pd.read_excel("../data/Ladesaeulenregister_SEP.xlsx")


def sort_by_plz_add_geometry(dfr, dfg): 
    dframe                  = dfr.copy()
    df_geo                  = dfg.copy()
    
    sorted_df               = dframe\
        .sort_values(by='PLZ')\
        .reset_index(drop=True)\
        .sort_index()
        
    sorted_df2              = sorted_df.merge(df_geo, on='PLZ', how ='left')
    sorted_df3              = sorted_df2.dropna(subset=['geometry'])
    
    sorted_df3['geometry']  = gpd.GeoSeries.from_wkt(sorted_df3['geometry'])
    ret                     = gpd.GeoDataFrame(sorted_df3, geometry='geometry')
    
    return ret



def preprocess_lstat(dfr, dfg):
    """Preprocessing dataframe from Ladesaeulenregister_SEP.xlsx"""
    dframe                  = dfr.copy()
    df_geo                  = dfg.copy()
    
    dframe2               	= dframe.loc[:,['Betreiber','Anzeigename (Karte)','Postleitzahl', 'Bundesland', 'Breitengrad', 'Längengrad', 'Nennleistung Ladeeinrichtung [kW]']]
    dframe2.rename(columns  = {'Betreiber':'stationOperator','Anzeigename (Karte)':'stationName',"Nennleistung Ladeeinrichtung [kW]":"KW", "Postleitzahl": "PLZ",  'Breitengrad':'Latitude', 'Längengrad':'Longitude', }, inplace = True)

    # Convert to string
    dframe2['Latitude']  = dframe2['Latitude'].astype(str)
    dframe2['Longitude']   = dframe2['Longitude'].astype(str)
    dframe2['stationOperator']   = dframe2['stationOperator'].astype(str)
    dframe2['stationName']   = dframe2['stationName'].astype(str)


    # Now replace the commas with periods
    dframe2['Latitude']  = dframe2['Latitude'].str.replace(',', '.')
    dframe2['Longitude']   = dframe2['Longitude'].str.replace(',', '.')

    dframe3                 = dframe2[(dframe2["Bundesland"] == 'Berlin') & 
                                            (dframe2["PLZ"] > 10115) &  
                                            (dframe2["PLZ"] < 14200)]
    
    ret = sort_by_plz_add_geometry(dframe3, df_geo)

    # Add an ID column with row numbers starting from 1
    ret['stationID'] = range(1, len(ret) + 1)

    # Add a blank column named 'Remarks' (all values set to empty strings)
    ret['ratingID'] = ''
    
    return ret


df = preprocess_lstat(df_lstat, df_geodat_plz)

df.to_csv("../data/ChargingStationData.csv.csv", index=False)

C:\Users\aka87578\AppData\Local\Temp\ipykernel_7008\2803414139.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_df3['geometry']  = gpd.GeoSeries.from_wkt(sorted_df3['geometry'])


In [41]:
df

,stationOperator,stationName,PLZ,Bundesland,Latitude,Longitude,KW,geometry,stationID,ratingID
0,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,10117,Berlin,52.51115,13.38952,30.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",1,
1,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,10117,Berlin,52.513083,13.389694,22.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",2,
2,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,10117,Berlin,52.515659,13.389829,44.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",3,
3,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,10117,Berlin,52.51897,13.390248,44.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",4,
4,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,10117,Berlin,52.520389,13.393959,44.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",5,
...,...,...,...,...,...,...,...,...,...,...
2883,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.479241,13.308733,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2874,
2884,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.476902,13.287843,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2875,
2885,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.479021,13.292588,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2876,
2886,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.484282,13.294078,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2877,
